# Requirements

In [ ]:
!pip install facenet_pytorch

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import MultiStepLR
from torchvision import datasets, transforms
from torch import optim
import numpy as np
import warnings
import zipfile
import torch
import uuid
import time
import cv2
import os

warnings.filterwarnings('ignore')

# Collect Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/ArshaFace.zip /content/

# Image Collection

In [ ]:
# root_dir = "/content/Train_Data"

# for i in range(300):
#   cap = cv2.VideoCapture(0)
#   print(f"Taking Picture number {i} ......")
#   time.sleep(5)
#   ret, frame = cap.read()
#   imgname = os.path.join(root_dir, f"_{i}.jpg")
#   cv2.imwrite(imgname, frame)
#   cv2.imshow('frame', frame)
#   time.sleep(3)

#   if cv2.waitKey(1) & 0xFF == ord('q'):
#     break

# cap.release()
# cv2.destroyAllWindows()

In [ ]:
# from IPython.display import display, Javascript
# from google.colab.output import eval_js
# from base64 import b64decode

# # def take_photo(filename='/content/Train_Data/photo.jpg', quality=0.8):
#   # root = "/content/Train_Data"
#   # filename = input("Image named as ")
#   # filename = os.path.join(root, filename + ".jpg")

# def take_photo(filename, quality=0.8):
#   js = Javascript('''
#     async function takePhoto(quality) {
#       const div = document.createElement('div');
#       const capture = document.createElement('button');
#       capture.textContent = 'Capture';
#       div.appendChild(capture);

#       const video = document.createElement('video');
#       video.style.display = 'block';
#       const stream = await navigator.mediaDevices.getUserMedia({video: true});

#       document.body.appendChild(div);
#       div.appendChild(video);
#       video.srcObject = stream;
#       await video.play();

#       // Resize the output to fit the video element.
#       google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

#       // Wait for Capture to be clicked.
#       await new Promise((resolve) => capture.onclick = resolve);

#       const canvas = document.createElement('canvas');
#       canvas.width = video.videoWidth;
#       canvas.height = video.videoHeight;
#       canvas.getContext('2d').drawImage(video, 0, 0);
#       stream.getVideoTracks()[0].stop();
#       div.remove();
#       return canvas.toDataURL('image/jpeg', quality);
#     }
#     ''')
#   display(js)
#   data = eval_js('takePhoto({})'.format(quality))
#   binary = b64decode(data.split(',')[1])
#   with open(filename, 'wb') as f:
#     f.write(binary)
#   return filename

In [ ]:
# from IPython.display import Image

# try:
#   for index in range(111,500):
#     print(f"Taking Picture number {index+1}/500 ......")
#     root = "/content/Train_Data"
#     filename = os.path.join(root, "Sample_" + str(index) + ".jpg")
#     path = take_photo(filename)
#     print('Saved to {}'.format(path))
#     print("-"*50)
#     time.sleep(2)

# except Exception as err:
#   print(str(err))



> Here comes the LFW dataset



In [ ]:
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz

--2024-08-15 06:22:43--  http://vis-www.cs.umass.edu/lfw/lfw.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180566744 (172M) [application/x-gzip]
Saving to: ‘lfw.tgz’

lfw.tgz             100%[===================>] 172.20M  1.07MB/s    in 3m 13s  

2024-08-15 06:25:56 (914 KB/s) - ‘lfw.tgz’ saved [180566744/180566744]



In [ ]:
!tar -xvzf lfw.tgz

Streaming output truncated to the last 5000 lines.
lfw/Luis_Sanchez/
lfw/Luis_Sanchez/Luis_Sanchez_0001.jpg
lfw/Juliette_Binoche/
lfw/Juliette_Binoche/Juliette_Binoche_0001.jpg
lfw/Joel_Todd/
lfw/Joel_Todd/Joel_Todd_0001.jpg
lfw/Allan_Wagner/
lfw/Allan_Wagner/Allan_Wagner_0001.jpg
lfw/Alejandro_Fernandez/
lfw/Alejandro_Fernandez/Alejandro_Fernandez_0001.jpg
lfw/Guillaume_Depardieu/
lfw/Guillaume_Depardieu/Guillaume_Depardieu_0001.jpg
lfw/Delphine_Chuillot/
lfw/Delphine_Chuillot/Delphine_Chuillot_0001.jpg
lfw/Paul_LeClerc/
lfw/Paul_LeClerc/Paul_LeClerc_0001.jpg
lfw/Eddie_Jordan/
lfw/Eddie_Jordan/Eddie_Jordan_0001.jpg
lfw/Lionel_Richie/
lfw/Lionel_Richie/Lionel_Richie_0001.jpg
lfw/Lionel_Richie/Lionel_Richie_0002.jpg
lfw/Jose_Viegas_Filho/
lfw/Jose_Viegas_Filho/Jose_Viegas_Filho_0001.jpg
lfw/Jose_Viegas_Filho/Jose_Viegas_Filho_0002.jpg
lfw/Roger_Mahony/
lfw/Roger_Mahony/Roger_Mahony_0001.jpg
lfw/Clay_Aiken/
lfw/Clay_Aiken/Clay_Aiken_0001.jpg
lfw/Clay_Aiken/Clay_Aiken_0002.jpg
lfw/Clay_Ai

In [ ]:
# Unzip my own data
!unzip /content/ArshaFace.zip -d /content/

Archive:  /content/ArshaFace.zip
  inflating: /content/dataset/Person 11/image_1.jpg  
  inflating: /content/dataset/Person 11/image_5.jpg  
  inflating: /content/dataset/Person 11/image_4.jpg  
  inflating: /content/dataset/Person 11/image_2.jpg  
  inflating: /content/dataset/Person 11/image_3.jpg  
  inflating: /content/dataset/Person 11/image_8.jpg  
  inflating: /content/dataset/Person 11/image_7.jpg  
  inflating: /content/dataset/Person 11/image_6.jpg  
  inflating: /content/dataset/Person 2/image_8.jpg  
  inflating: /content/dataset/Person 2/image_6.jpg  
  inflating: /content/dataset/Person 2/image_4.jpg  
  inflating: /content/dataset/Person 2/image_7.jpg  
  inflating: /content/dataset/Person 2/image_9.jpg  
  inflating: /content/dataset/Person 2/image_1.jpg  
  inflating: /content/dataset/Person 2/image_10.jpg  
  inflating: /content/dataset/Person 2/image_2.jpg  
  inflating: /content/dataset/Person 2/image_5.jpg  
  inflating: /content/dataset/Person 2/image_3.jpg  
  in

In [ ]:
# Delete unnecessary directories (all of them expect Person 4)
!rm -r "/content/dataset/Person 14"

#### Make Sure to rename the data directory to this format:
- /content/dataset/Person1


In [ ]:
# Should be 90
os.rename("/content/dataset/Person 4", "/content/dataset/Person1")
len(os.listdir("/content/dataset/Person1"))

90

#### Now move all the LFW faces into another folder called Person2, as here we are only training the model to verify one face

In [ ]:
from tqdm import tqdm
import shutil
import random

root_path = "/content/lfw"
dest_path = "/content/dataset/Person2"
os.makedirs(dest_path, exist_ok=True)

for person in tqdm(os.listdir(root_path)):
  person_path = os.path.join(root_path, person)
  print(f"{person} images are being moved ...")
  if len(os.listdir(root_path)) >= 1:
    sample = random.choice(os.listdir(person_path))
    src = os.path.join(person_path, sample)
    dest = os.path.join(dest_path, sample)
    shutil.move(src, dest)
    print("-"*100)

  else:
    print("No Images present in the folder!")

  # for image in os.listdir(person_path):
  #   src = os.path.join(person_path, image)
  #   dest = os.path.join(dest_path, image)
  #   shutil.move(src, dest)
  # print("-"*100)




  1%|          | 30/5749 [00:00<00:19, 290.94it/s]

Lisa_Stone images are being moved ...
----------------------------------------------------------------------------------------------------
John_Cusack images are being moved ...
----------------------------------------------------------------------------------------------------
Billy_Crawford images are being moved ...
----------------------------------------------------------------------------------------------------
Jeffrey_Immelt images are being moved ...
----------------------------------------------------------------------------------------------------
Uday_Hussein images are being moved ...
----------------------------------------------------------------------------------------------------
Patrick_Stewart images are being moved ...
----------------------------------------------------------------------------------------------------
Abel_Aguilar images are being moved ...
----------------------------------------------------------------------------------------------------
Leslie_Mo

  2%|▏         | 90/5749 [00:00<00:19, 284.53it/s]

----------------------------------------------------------------------------------------------------
Paula_Radcliffe images are being moved ...
----------------------------------------------------------------------------------------------------
Brendan_Fraser images are being moved ...
----------------------------------------------------------------------------------------------------
Caroline_Kennedy images are being moved ...
----------------------------------------------------------------------------------------------------
Massoud_Barzani images are being moved ...
----------------------------------------------------------------------------------------------------
Jimmy_Carter images are being moved ...
----------------------------------------------------------------------------------------------------
Kristin_Scott_Thomas images are being moved ...
----------------------------------------------------------------------------------------------------
Albert_Costa images are being mov

  3%|▎         | 148/5749 [00:00<00:19, 283.34it/s]

...

----------------------------------------------------------------------------------------------------
Jose_Canseco_Sr images are being moved ...
----------------------------------------------------------------------------------------------------
Gene_Robinson images are being moved ...
----------------------------------------------------------------------------------------------------
Kenneth_Bowersox images are being moved ...
----------------------------------------------------------------------------------------------------
William_Harrison images are being moved ...
----------------------------------------------------------------------------------------------------
Sok_An images are being moved ...
----------------------------------------------------------------------------------------------------
Helena_Schneider images are being moved ...
----------------------------------------------------------------------------------------------------
Ali_Adbul_Karim_Madani images are being mov

 99%|█████████▉| 5702/5749 [00:23<00:00, 179.29it/s]


Marsha_Thomason images are being moved ...
----------------------------------------------------------------------------------------------------
Brendan_Gaughan images are being moved ...
----------------------------------------------------------------------------------------------------
Rogelio_Montemayor images are being moved ...
----------------------------------------------------------------------------------------------------
Jose_Woldenberg images are being moved ...
----------------------------------------------------------------------------------------------------
Tina_Andrews images are being moved ...
----------------------------------------------------------------------------------------------------
Angela_Mascia-Frye images are being moved ...
----------------------------------------------------------------------------------------------------
Ruben_Studdard images are being moved ...
------------------------------------------------------------------------------------------

100%|██████████| 5749/5749 [00:24<00:00, 237.44it/s]


----------------------------------------------------------------------------------------------------
Amber_Frey images are being moved ...
----------------------------------------------------------------------------------------------------
Martin_Landau images are being moved ...
----------------------------------------------------------------------------------------------------
Trevor_McDonald images are being moved ...
----------------------------------------------------------------------------------------------------
Carol_Niedermayer images are being moved ...
----------------------------------------------------------------------------------------------------
Jake_Gyllenhaal images are being moved ...
----------------------------------------------------------------------------------------------------
Giulio_Andreotti images are being moved ...
----------------------------------------------------------------------------------------------------
Bill_Simon images are being moved ...


In [ ]:
# Should be about 5.7k
len(os.listdir("/content/dataset/Person1"))

90

# Data Augmentation

In [ ]:
import Data_and_Label_Augmentation as DataGen

#### Initialize Image Custom Augmentation object
My_data = DataGen.Image_Custom_Augmentation(
                                    # SP_intensity=0.010,
                                    SP_intensity=False,
                                    CWRO_Key=20,
                                    CCWRO_Key=20,
                                    Br_intensity=25,
                                    H_Key = True,
                                    V_Key = False,
                                    HE_Key= False,
                                    GaussianBlur_KSize = 9,
                                    Random_Translation = True,
                                    Scaling_Range = (0.90, 1.10),
                                    Img_res=512
                                    )

#### Generate augmented data
My_data.Generate_Data(input_path="/content/dataset/Person1", output_path="/content/dataset/Person1")

100%|██████████| 90/90 [01:46<00:00,  1.18s/it]


# Fine-Tune

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0




> Error Handling:

A hidden .ipynb file causes error, so make sure to run the cells bellow with respect to the directory of your dataset



In [ ]:
!rm -R /content/dataset/.ipynb_checkpoints

rm: cannot remove '/content/dataset/.ipynb_checkpoints': No such file or directory


In [ ]:
!rm -R /content/dataset/Person1/.ipynb_checkpoints

rm: cannot remove '/content/dataset/Person1/.ipynb_checkpoints': No such file or directory


In [ ]:
!rm -R /content/dataset/Person2/.ipynb_checkpoints

rm: cannot remove '/content/dataset/Person2/.ipynb_checkpoints': No such file or directory


In [ ]:
len(os.listdir("/content/dataset/Person2"))

5749

In [ ]:
data_dir = '/content/dataset'

batch_size = 1
epochs = 120
workers = 0 if os.name == 'nt' else 8

In [ ]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [ ]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
# dataset = datasets.ImageFolder(data_dir)
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]

loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
  try:
      mtcnn(x, save_path=y)
      print('\rimage number {} of {}'.format(i + 1, len(loader)), end='')

  except Exception as e:
    print(f"the problem is on x: {x}\n")
    print(f"the problem is on y: {y}")

# # Debugging the corrupted images

# valid_indices = []
# invalid_indices = []

# for i, (x, y) in tqdm(enumerate(loader)):
#     try:
#         # Process images with MTCNN
#         boxes, probs = mtcnn.detect(x)
#         # Filter out images where no face is detected and print the image name
#         for idx, box in enumerate(boxes):
#             if box is not None:
#                 valid_indices.append(dataset.samples[idx][0])
#             else:
#                 # print(f"\nNo face detected in image: {dataset.samples[idx][0]}")
#                 invalid_indices.append(dataset.samples[idx][0])

#     except Exception as e:
#         print(f"\nError in batch {i + 1}: {e}")
#         continue

# print(invalid_indices)

    #     x_valid = [x[i] for i in valid_indices]
    #     y_valid = [y[i] for i in valid_indices]

    #     if x_valid:
    #         mtcnn(x_valid, save_path=y_valid)

    #     print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')

    # except Exception as e:
    #     print(f"\nError in batch {i + 1}: {e}")
    #     continue

# Remove mtcnn to reduce GPU memory usage
del mtcnn

image number 6559 of 6559

In [ ]:
# Just to check how many images the MTCNN failed to get
print(len(os.listdir("/content/dataset_cropped/Person1")))

763


In [ ]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [ ]:
!rm -R /content/dataset_cropped/.ipynb_checkpoints

rm: cannot remove '/content/dataset_cropped/.ipynb_checkpoints': No such file or directory


In [ ]:
!rm -R /content/dataset_cropped/Person1/.ipynb_checkpoints

rm: cannot remove '/content/dataset_cropped/Person1/.ipynb_checkpoints': No such file or directory


In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
# loss_fn = torch.nn.BCEWithLogitsLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# batch size
BS = 16
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110])
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)


trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

dataset = datasets.ImageFolder("/content/dataset_cropped", transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.75 * len(img_inds))]
val_inds = img_inds[int(0.75 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=BS,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=BS,
    sampler=SubsetRandomSampler(val_inds)
)

In [ ]:
# def compute_accuracy(y_pred, y_true):
#     y_pred_binary = (y_pred > 0.5).float()
#     correct = (y_pred_binary == y_true).sum().item()
#     total = y_true.size(0)
#     accuracy = correct / total
#     return accuracy

In [ ]:
# # Training loop
# for epoch in range(epochs):
#     print('\nEpoch {}/{}'.format(epoch + 1, epochs))

#     # Training loop
#     resnet.train()
#     train_loss = 0.0
#     train_correct = 0
#     train_total = 0

#     resnet.train()
#     for x, y in train_loader:
#         y = y.float().unsqueeze(1).to(device)
#         y_pred = resnet(x.to(device))
#         loss_batch = loss_fn(y_pred, y)

#         # Backpropagation and optimization
#         optimizer.zero_grad()
#         loss_batch.backward()
#         optimizer.step()

#         # Loss Report
#         train_loss += loss_batch.item() * x.size(0)
#         y_pred_binary = (y_pred > 0.5).float()
#         train_correct += (y_pred_binary == y).sum().item()
#         train_total += y.size(0)

#     train_loss /= train_total
#     train_accuracy = train_correct / train_total

#     print(f"Train Loss: {train_loss:.4f}")
#     print(f"Train Accuracy: {train_accuracy:.4f}")


#     # Validation Loop
#     resnet.eval()
#     val_loss = 0.0
#     val_correct = 0
#     val_total = 0

#     with torch.no_grad():
#       for x, y in val_loader:
#           y = y.float().unsqueeze(1).to(device)
#           y_pred = resnet(x.to(device))
#           loss_batch = loss_fn(y_pred, y)

#           val_loss += loss_batch.item() * x.size(0)
#           y_pred_binary = (y_pred > 0.5).float()
#           val_correct += (y_pred_binary == y).sum().item()
#           val_total += y.size(0)

#     val_loss /= val_total
#     val_accuracy = val_correct / val_total
#     print(f"Validation Loss: {val_loss:.4f}")
#     print(f"Validation Accuracy: {val_accuracy:.4f}")
#     print("="*100)

#     scheduler.step(val_loss)



In [ ]:
# writer = SummaryWriter()
# writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    # writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        # writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        # writer=writer
    )

# writer.close()



Initial
----------
Valid |   102/102  | loss:    0.6938 | fps:  208.5708 | acc:    0.5895   

Epoch 1/120
----------
Train |   306/306  | loss:    0.3050 | fps:  131.3605 | acc:    0.9101   
Valid |   102/102  | loss:    0.1576 | fps:  275.2091 | acc:    0.9600   

Epoch 2/120
----------
Train |   306/306  | loss:    0.1054 | fps:  127.8627 | acc:    0.9653   
Valid |   102/102  | loss:    0.0951 | fps:  274.2138 | acc:    0.9651   

Epoch 3/120
----------
Train |   306/306  | loss:    0.0591 | fps:  129.5390 | acc:    0.9796   
Valid |   102/102  | loss:    0.0311 | fps:  207.1143 | acc:    0.9920   

Epoch 4/120
----------
Train |   306/306  | loss:    0.1033 | fps:  128.9949 | acc:    0.9669   
Valid |   102/102  | loss:    0.0315 | fps:  277.4977 | acc:    0.9908   

Epoch 5/120
----------
Train |   306/306  | loss:    0.1485 | fps:  128.0296 | acc:    0.9551   
Valid |   102/102  | loss:    0.1317 | fps:  209.3125 | acc:    0.9626   

Epoch 6/120
----------
Train |   306/306  | 

In [ ]:
torch.save(resnet.state_dict(), '/content/Face_Verification_v4.pth')

In [ ]:
!cp -r /content/Face_Verification_v4.pth /content/drive/MyDrive

In [ ]:
# import re

# file = open("/content/viz.txt", 'r')
# for line in file.readlines():
#   if line.startswith("Train"):
#     a = re.search(r'\b(loss)\b', line)
#     print(f"loss index in text: {a.start()}")
#     b = re.search(r'\b(acc)\b', line)
#     print(f"accuracy index in text: {b.start()}")

# file.close()

loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56
loss index in text: 21
accuracy index in text: 56


In [ ]:
# from tqdm import tqdm
# file = open("/content/viz.txt", 'r')

# train_loss = []
# train_acc = []

# for line in tqdm(file.readlines()):
#   if line.startswith("Train"):
#     # print(line[30:36]) # Train Loss Values
#     # print(line[64:70]) # Train Accuracy Values
#     train_loss.append(float(line[30:36]))
#     train_acc.append(float(line[64:70]))

# file.close()

100%|██████████| 151/151 [00:00<00:00, 467409.52it/s]


In [ ]:
# from tqdm import tqdm
# file = open("/content/viz.txt", 'r')

# val_loss = []
# val_acc = []

# for line in tqdm(file.readlines()):
#   if line.startswith("Valid"):
#     # print(line[30:36]) # Valid Loss Values
#     # print(line[64:70]) # Valid Accuracy Values
#     val_loss.append(float(line[30:36]))
#     val_acc.append(float(line[64:70]))

# file.close()

100%|██████████| 151/151 [00:00<00:00, 336882.93it/s]
